In [45]:
import pandas as pd, numpy as np

cpu_df = pd.read_csv('datasets/CPUs_Edited.csv')
cpu_df.head()

CPU Name  Cores CPU Mark - Thread Mark -
0                AMD 3015e      2      2,720         1,391
1                AMD 3020e      2      2,493         1,428
2  AMD A10 Micro-6700T APU      4      1,345           700
3    AMD A10 PRO-7350B APU      4      1,910           911
4        AMD A10-4600M APU      2      1,923         1,067

In [46]:
gpu_df = pd.read_csv('datasets/GPUs_Edited.csv')
gpu_df.head()

Videocard Name G3D Mark - G2D Mark -
0  128 DDR Radeon 9700 TX w/TV-Out         44        152
1          128 DDR Radeon 9800 Pro         62        191
2        128MB DDR Radeon 9800 Pro         66        175
3             128MB RADEON X600 SE         49        229
4                             15FF      8,229        538

In [47]:
df = pd.read_csv('datasets/cleaned_df.csv')
df.head()

laptop_ID Company      Product   TypeName  Inches  \
0          1   Apple  MacBook Pro  Ultrabook    13.3   
1          2   Apple  Macbook Air  Ultrabook    13.3   
2          3      HP       250 G6   Notebook    15.6   
3          4   Apple  MacBook Pro  Ultrabook    15.4   
4          5   Apple  MacBook Pro  Ultrabook    13.3   

                     ScreenResolution                         Cpu   Ram  \
0  IPS Panel Retina Display 2560x1600        Intel Core i5 2.3GHz   8GB   
1                            1440x900        Intel Core i5 1.8GHz   8GB   
2                   Full HD 1920x1080  Intel Core i5 7200U 2.5GHz   8GB   
3  IPS Panel Retina Display 2880x1800        Intel Core i7 2.7GHz  16GB   
4  IPS Panel Retina Display 2560x1600        Intel Core i5 3.1GHz   8GB   

                Memory                           Gpu  OpSys  Weight  \
0            128GB SSD  Intel Iris Plus Graphics 640  macOS  1.37kg   
1  128GB Flash Storage        Intel HD Graphics 6000  macOS  1.34kg   
2            256GB SSD         Intel HD Graphics 620  No OS  1.86kg   
3            512GB SSD            AMD Radeon Pro 455  macOS  1.83kg   
4            256GB SSD  Intel Iris Plus Graphics 650  macOS  1.37kg   

   Price_euros  
0      1339.69  
1       898.94  
2       575.00  
3      2537.45  
4      1803.60

In [48]:
df.keys()

Index(['laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
       'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight',
       'Price_euros'],
      dtype='object')

In [49]:
cpu_df.keys()

Index(['CPU Name', 'Cores', 'CPU Mark -', 'Thread Mark -'], dtype='object')

In [50]:
#preprocess Product column
df.drop('laptop_ID', axis = 1, inplace = True)
df.drop('Product', axis = 1, inplace = True)
# df.drop('Price_euros', axis = 1, inplace = True)

In [51]:
df.head()

Company   TypeName  Inches                    ScreenResolution  \
0   Apple  Ultrabook    13.3  IPS Panel Retina Display 2560x1600   
1   Apple  Ultrabook    13.3                            1440x900   
2      HP   Notebook    15.6                   Full HD 1920x1080   
3   Apple  Ultrabook    15.4  IPS Panel Retina Display 2880x1800   
4   Apple  Ultrabook    13.3  IPS Panel Retina Display 2560x1600   

                          Cpu   Ram               Memory  \
0        Intel Core i5 2.3GHz   8GB            128GB SSD   
1        Intel Core i5 1.8GHz   8GB  128GB Flash Storage   
2  Intel Core i5 7200U 2.5GHz   8GB            256GB SSD   
3        Intel Core i7 2.7GHz  16GB            512GB SSD   
4        Intel Core i5 3.1GHz   8GB            256GB SSD   

                            Gpu  OpSys  Weight  Price_euros  
0  Intel Iris Plus Graphics 640  macOS  1.37kg      1339.69  
1        Intel HD Graphics 6000  macOS  1.34kg       898.94  
2         Intel HD Graphics 620  No OS  1.86kg       575.00  
3            AMD Radeon Pro 455  macOS  1.83kg      2537.45  
4  Intel Iris Plus Graphics 650  macOS  1.37kg      1803.60

In [52]:
X_df = df.drop('Price_euros',axis =1).copy()

In [65]:
pd.options.display.max_columns = None

# All in One Preprocess method 

In [53]:
def preprocess(X_df):
    
    global gpu_df
    global cpu_df
    import re
    import difflib
    
    df = X_df.copy(deep = True)
    
    #preprocess Company column
    keys = ['Mediacom', 'Microsoft', 'Razer', 'Google'] + ['Xiaomi','Chuwi','Huawei'] + ['Vero','Fujitsu']
    values = ['other (european/American)'] * 4 + ['other (Chinease/Korean)'] * 3 + ['other'] * 2
    replacement_dict = dict(zip(keys, values))
    df.loc[:, 'Company'].replace(replacement_dict, inplace = True)
    values = ['other (european/American)', 'other (Chinease/Korean)', 'other']
    
    categories = ['Apple', 'HP', 'Acer', 'Asus', 'Dell', 'Lenovo',
                  'other (Chinease/Korean)', 'MSI', 'other (european/American)',
                  'Toshiba', 'Samsung', 'LG']
    for cat in categories:
        df[f'Company: {cat}'] = df['Company'].apply(lambda x: 1 if x == cat else 0)

    # df[[f'Company: {x}' for x in categories]] = df['Company'].apply(lambda x: pd.Series([1 if y == x else 0 for y in categories]))
    df.drop('Company', axis = 1, inplace = True)
    
    #preprocess TypeName column
    df.loc[:, 'TypeName'].replace({'Netbook': 'Notebook'}, inplace = True)
    categories = ['Ultrabook', 'Gaming', '2 in 1 Convertible', 'Workstation']
    
    
    df[[f'Type: {x}' for x in categories]] = df['TypeName'].apply(lambda x: pd.Series([1 if y == x else 0 for y in categories]))
    df.drop('TypeName', axis = 1, inplace = True)
    
    #preprocessing Memory column
    categories = ['SSD', 'HDD', 'Flash', 'Hybrid']
    df[[f'Memory: {x}' for x in categories]] = df['Memory'].apply(lambda x: pd.Series([1 if y in x else 0 for y in categories]))
    df[['Memory (GB): Primary', 'Memory (GB): Secondary']] = df['Memory'].apply(lambda x: pd.Series([int(y.replace('TB', '')) * 1024 if 'TB' in y else int(y.replace('GB', '')) for y in (re.findall(r'-?\d+\w+\.?\d*\w+', x) if len(re.findall(r'-?\d+\w+\.?\d*\w+', x)) == 2 else re.findall(r'-?\d+\w+\.?\d*\w+', x) + ['0GB'])])).fillna(0).astype('int')
    df.drop('Memory', axis = 1, inplace = True)
    
    #preprocess Ram column
    df['Ram Size (GB)'] = df['Ram'].str.replace('GB', '').astype('int')
    df.drop('Ram', axis = 1, inplace = True)
    
    #preprocess Weight column
    df['Weight (kg)'] = df['Weight'].str.replace('kg', '').astype('float16')
    df.drop('Weight', axis = 1, inplace = True)
    
    #preprocess OpSys column
    df.loc[:, 'OpSys'].replace({'Mac OS X': 'macOS', 'Windows 10 S': 'Windows 10'}, inplace = True)
    categories = ['macOS', 'Windows 10', 'Linux', 'Android', 'Chrome OS', 'Windows 7']
    df[[f'OpSys: {x}' for x in categories]] = df['OpSys'].apply(lambda x: pd.Series([1 if y == x else 0 for y in categories]))
    df.drop('OpSys', axis = 1, inplace = True)
    
    #preprocess Inches column
    df['Screen Size (inch)'] = df['Inches']
    df.drop('Inches', axis = 1, inplace = True)
    
    #preprocess ScreenResolution column
    screen_features = ['IPS', 'Touchscreen', 'Retina']
    
    df[['Screen Resolution (px): Horizontal', 
        'Screen Resolution (px): Vertical', 
        'Screen Features: IPS', 
        'Screen Features: Touchscreen', 
        'Screen Features: Retina Display']] = df['ScreenResolution'].apply(
        lambda x: pd.Series([int(y) for y in x.split()[-1].split('x')] + [1 if y in x else 0 for y in screen_features]))
    
    df.drop('ScreenResolution', axis = 1, inplace = True)
    
    #preprocess CPUs and GPUs
    df['gpu_name'] = [name + (50 -len(name)) * " " for name in df.Gpu]
    df['cpu_name'] = [name + (50 -len(name)) * " " for name in df.Cpu]
    
    gpu_df['Videocard Name'] = [name + (50 -len(name)) * " " for name in gpu_df['Videocard Name']]
    cpu_df['CPU Name'] =       [name + (50 -len(name)) * " " for name in cpu_df['CPU Name']]
    
    for idx_laptop, row_laptop in df.iterrows():  
            gpu_key = difflib.get_close_matches(df.loc[idx_laptop,'gpu_name'], gpu_df['Videocard Name'] ,n=1)[0]
            df.loc[idx_laptop,'gpu_name_key'] = gpu_key

            cpu_key = difflib.get_close_matches(df.loc[idx_laptop,'cpu_name'], cpu_df['CPU Name'] ,n=1)[0]
            df.loc[idx_laptop,'cpu_name_key'] = cpu_key
    
    df['gpu_G3D_Mark'] = 0
    df['gpu_G2D_Mark'] = 0
    df['cpu_Thread_Mark'] = 0
    df['cpu_mark'] = 0
    df['cpu_cores'] = 0
    
    gpu_df['G3D Mark -']    = gpu_df['G3D Mark -'].str.replace(",","")
    gpu_df['G2D Mark -']    = gpu_df['G2D Mark -'].str.replace(",","")
    cpu_df['Cores']         = cpu_df['Cores']
    cpu_df['CPU Mark -']    = cpu_df['CPU Mark -'].str.replace(",","")
    cpu_df['Thread Mark -'] = cpu_df['Thread Mark -'].str.replace(",","")

    for idx_laptop, row_laptop in df.iterrows():

        gpu_key = df.loc[idx_laptop, 'gpu_name_key'] 
        df.loc[idx_laptop,'gpu_G3D_Mark'] = gpu_df.query("{0} == @gpu_key".format('`Videocard Name`')).iloc[0]['G3D Mark -'].replace(",","")
        df.loc[idx_laptop,'gpu_G2D_Mark'] = gpu_df.query("{0} == @gpu_key".format('`Videocard Name`')).iloc[0]['G2D Mark -'].replace(",","")

        cpu_key = df.loc[idx_laptop,'cpu_name_key'] 
        df.loc[idx_laptop,'cpu_Thread_Mark'] = cpu_df.query("{0} == @cpu_key".format('`CPU Name`')).iloc[0]['Thread Mark -'].replace(",","")
        df.loc[idx_laptop,'cpu_mark'] = cpu_df.query("{0} == @cpu_key".format('`CPU Name`')).iloc[0]['CPU Mark -'].replace(",","")
        df.loc[idx_laptop,'cpu_cores'] = cpu_df.query("{0} == @cpu_key".format('`CPU Name`')).iloc[0]['Cores']


    df['gpu_G3D_Mark'] = df['gpu_G3D_Mark'].astype('int')
    df['gpu_G2D_Mark'] = df['gpu_G2D_Mark'].astype('int')
    df['cpu_Thread_Mark'] = df['cpu_Thread_Mark'].astype('int')
    df['cpu_mark'] = df['cpu_mark'].astype('int')

    #preprocess CPU and GPU columns
    df.drop(['Cpu', 'Gpu','cpu_name','gpu_name','cpu_name_key','gpu_name_key'], axis = 1, inplace = True)
    
    return df

X_temp = preprocess(X_df)
X_temp.head()

Company: Apple  Company: HP  Company: Acer  Company: Asus  Company: Dell  \
0               1            0              0              0              0   
1               1            0              0              0              0   
2               0            1              0              0              0   
3               1            0              0              0              0   
4               1            0              0              0              0   

   Company: Lenovo  Company: other (Chinease/Korean)  Company: MSI  \
0                0                                 0             0   
1                0                                 0             0   
2                0                                 0             0   
3                0                                 0             0   
4                0                                 0             0   

   Company: other (european/American)  Company: Toshiba  ...  \
0                                   0                 0  ...   
1                                   0                 0  ...   
2                                   0                 0  ...   
3                                   0                 0  ...   
4                                   0                 0  ...   

   Screen Resolution (px): Horizontal  Screen Resolution (px): Vertical  \
0                                2560                              1600   
1                                1440                               900   
2                                1920                              1080   
3                                2880                              1800   
4                                2560                              1600   

   Screen Features: IPS  Screen Features: Touchscreen  \
0                     1                             0   
1                     0                             0   
2                     0                             0   
3                     1                             0   
4                     1                             0   

   Screen Features: Retina Display  gpu_G3D_Mark  gpu_G2D_Mark  \
0                                1          1429           563   
1                                0          1037           247   
2                                0           922           218   
3                                1          3113           705   
4                                1          1429           563   

   cpu_Thread_Mark  cpu_mark  cpu_cores  
0             3712     23580          4  
1             3604     19052          6  
2             1780      3413          2  
3             3747     27545          6  
4             3712     23580          4  

[5 rows x 41 columns]

In [54]:
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

In [55]:
# X_df =  df.drop('Price_euros', axis = 1)
y = df['Price_euros']

In [56]:
# pipeline = Pipeline(steps = [('preprocessor', FunctionTransformer(preprocess)),
#                              ('scaler', StandardScaler()), 
#                              ('model', RandomForestRegressor(random_state = 42))])

# pipeline.fit(X_df, y = y)

# pipeline.predict(X_df.loc[[0]])

In [57]:
# import joblib
# joblib.dump(pipeline, 'pipeline.h5')

# Randomized Grid Search

In [58]:
from sklearn.model_selection import RandomizedSearchCV

In [59]:
### CANCELED ==========================================

# rscv = RandomizedSearchCV(pipeline, 
#                           n_iter = 10, 
#                           scoring = 'r2',
#                           cv = 5,
#                           return_train_score = True,
#                           verbose = 4,
#                           n_jobs = -1,
#                           param_distributions = {
#                               'model__n_estimators': range(50, 150, 10),
#                               'model__max_depth': range(10, 40, 5),
#                               'model__min_samples_leaf': range(5, 26, 5),
#                               'model__min_samples_split': range(10, 41, 5)
#                           })

In [60]:
# rscv.fit(X_df, y)

In [61]:
# rscv.best_estimator_

# rscv.best_params_

# rscv.best_score_

<font color = 'blue'><h2> running a grid search on the Preprocess function results in exteremely long processing time with waisted computing effort. therefore, we're going to pick the best parameters for the Model outside of the Pipeline and rebuild it</h2></font>

# Parameter Tuning

In [66]:
# X_temp = X_df.copy()
X_temp.head()

Company: Apple  Company: HP  Company: Acer  Company: Asus  Company: Dell  \
0               1            0              0              0              0   
1               1            0              0              0              0   
2               0            1              0              0              0   
3               1            0              0              0              0   
4               1            0              0              0              0   

   Company: Lenovo  Company: other (Chinease/Korean)  Company: MSI  \
0                0                                 0             0   
1                0                                 0             0   
2                0                                 0             0   
3                0                                 0             0   
4                0                                 0             0   

   Company: other (european/American)  Company: Toshiba  Company: Samsung  \
0                                   0                 0                 0   
1                                   0                 0                 0   
2                                   0                 0                 0   
3                                   0                 0                 0   
4                                   0                 0                 0   

   Company: LG  Type: Ultrabook  Type: Gaming  Type: 2 in 1 Convertible  \
0            0                1             0                         0   
1            0                1             0                         0   
2            0                0             0                         0   
3            0                1             0                         0   
4            0                1             0                         0   

   Type: Workstation  Memory: SSD  Memory: HDD  Memory: Flash  Memory: Hybrid  \
0                  0            1            0              0               0   
1                  0            0            0              1               0   
2                  0            1            0              0               0   
3                  0            1            0              0               0   
4                  0            1            0              0               0   

   Memory (GB): Primary  Memory (GB): Secondary  Ram Size (GB)  Weight (kg)  \
0                   128                       0              8     1.370117   
1                   128                       0              8     1.339844   
2                   256                       0              8     1.860352   
3                   512                       0             16     1.830078   
4                   256                       0              8     1.370117   

   OpSys: macOS  OpSys: Windows 10  OpSys: Linux  OpSys: Android  \
0             1                  0             0               0   
1             1                  0             0               0   
2             0                  0             0               0   
3             1                  0             0               0   
4             1                  0             0               0   

   OpSys: Chrome OS  OpSys: Windows 7  Screen Size (inch)  \
0                 0                 0                13.3   
1                 0                 0                13.3   
2                 0                 0                15.6   
3                 0                 0                15.4   
4                 0                 0                13.3   

   Screen Resolution (px): Horizontal  Screen Resolution (px): Vertical  \
0                                2560                              1600   
1                                1440                               900   
2                                1920                              1080   
3                                2880                              1800   
4                                2560                              1600   

   S

In [67]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_temp)
X_scaled

array([[ 7.77051815, -0.5111286 , -0.29469738, ...,  3.96502868,
         5.74530219,  1.1216814 ],
       [ 7.77051815, -0.5111286 , -0.29469738, ...,  3.74955298,
         4.38890371,  3.00921055],
       [-0.12869155,  1.95645479, -0.29469738, ...,  0.11040785,
        -0.29588333, -0.76584775],
       ...,
       [-0.12869155, -0.5111286 , -0.29469738, ..., -2.31568891,
        -1.14123591, -0.76584775],
       [-0.12869155,  1.95645479, -0.29469738, ..., -0.03124747,
        -0.33572454, -0.76584775],
       [-0.12869155, -0.5111286 , -0.29469738, ..., -2.31568891,
        -1.14123591, -0.76584775]])

In [97]:
from xgboost import XGBRegressor

In [159]:
model = XGBRegressor(random_state = 42)

rscv = RandomizedSearchCV(model, 
                          n_iter = 100, 
                          scoring = 'r2',
                          cv = 10,
                          return_train_score = True,
                          verbose = 4,
                          n_jobs = -1,
                          param_distributions = {
                              'n_estimators': range(80, 120, 2),
                              'max_depth': range(5, 20),
                              'min_samples_leaf': range(10, 30, 2),
                              'min_samples_split': range(10, 30, 2),
                              'colsample_bytree': np.arange(0.2, 1.0, 0.1),
                              'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
                              'subsample': np.arange(0.3, 1.0, 0.1),
                              'learning_rate': np.arange(0.05, 1.0)
                          })

In [160]:
rscv.fit(X_scaled, y)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[22:56:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "min_samples_leaf", "min_samples_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




RandomizedSearchCV(cv=10,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          enable_categorical=False, gamma=None,
                                          gpu_id=None, importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints...
                   param_distributions={'colsample_bylevel': array([0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                        'colsample_bytree': array([0.2, 0.3, 0.4, 0.5, 0.6,

In [209]:
xgboost_model = rscv.best_estimator_

In [220]:
xgboost_model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.7,
             colsample_bynode=1, colsample_bytree=0.4000000000000001,
             enable_categorical=False, gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.05, max_delta_step=0,
             max_depth=9, min_child_weight=1, min_samples_leaf=12,
             min_samples_split=20, missing=nan, monotone_constraints='()',
             n_estimators=118, n_jobs=12, num_parallel_tree=1, predictor='auto',
             random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=0.9000000000000001, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [210]:
import joblib

xgboost_model.save_model("xgboost.h5")

In [165]:
rscv.best_params_

{'subsample': 0.9000000000000001,
 'n_estimators': 118,
 'min_samples_split': 20,
 'min_samples_leaf': 12,
 'max_depth': 9,
 'learning_rate': 0.05,
 'colsample_bytree': 0.4000000000000001,
 'colsample_bylevel': 0.7}

In [166]:
rscv.best_score_

0.8640878036689458

final score for our model that will be used for deployment

# final Pipeline

Now we create a new pipeline with the best parameters found in the grid search

In [73]:
X_df.head()

Company   TypeName  Inches                    ScreenResolution  \
0   Apple  Ultrabook    13.3  IPS Panel Retina Display 2560x1600   
1   Apple  Ultrabook    13.3                            1440x900   
2      HP   Notebook    15.6                   Full HD 1920x1080   
3   Apple  Ultrabook    15.4  IPS Panel Retina Display 2880x1800   
4   Apple  Ultrabook    13.3  IPS Panel Retina Display 2560x1600   

                          Cpu   Ram               Memory  \
0        Intel Core i5 2.3GHz   8GB            128GB SSD   
1        Intel Core i5 1.8GHz   8GB  128GB Flash Storage   
2  Intel Core i5 7200U 2.5GHz   8GB            256GB SSD   
3        Intel Core i7 2.7GHz  16GB            512GB SSD   
4        Intel Core i5 3.1GHz   8GB            256GB SSD   

                            Gpu  OpSys  Weight  
0  Intel Iris Plus Graphics 640  macOS  1.37kg  
1        Intel HD Graphics 6000  macOS  1.34kg  
2         Intel HD Graphics 620  No OS  1.86kg  
3            AMD Radeon Pro 455  macOS  1.83kg  
4  Intel Iris Plus Graphics 650  macOS  1.37kg

In [221]:
from sklearn.base import BaseEstimator, TransformerMixin

class feature_transformer(BaseEstimator,TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self,X,y=None):
        return self
    
    def transform(self, X):
        
        return preprocess(X)

In [223]:
feature_transformer().transform(X_df.loc[[0]])

Company: Apple  Company: HP  Company: Acer  Company: Asus  Company: Dell  \
0               1            0              0              0              0   

   Company: Lenovo  Company: other (Chinease/Korean)  Company: MSI  \
0                0                                 0             0   

   Company: other (european/American)  Company: Toshiba  Company: Samsung  \
0                                   0                 0                 0   

   Company: LG  Type: Ultrabook  Type: Gaming  Type: 2 in 1 Convertible  \
0            0                1             0                         0   

   Type: Workstation  Memory: SSD  Memory: HDD  Memory: Flash  Memory: Hybrid  \
0                  0            1            0              0               0   

   Memory (GB): Primary  Memory (GB): Secondary  Ram Size (GB)  Weight (kg)  \
0                   128                       0              8     1.370117   

   OpSys: macOS  OpSys: Windows 10  OpSys: Linux  OpSys: Android  \
0             1                  0             0               0   

   OpSys: Chrome OS  OpSys: Windows 7  Screen Size (inch)  \
0                 0                 0                13.3   

   Screen Resolution (px): Horizontal  Screen Resolution (px): Vertical  \
0                                2560                              1600   

   Screen Features: IPS  Screen Features: Touchscreen  \
0                     1                             0   

   Screen Features: Retina Display  gpu_G3D_Mark  gpu_G2D_Mark  \
0                                1          1429           563   

   cpu_Thread_Mark  cpu_mark  cpu_cores  
0             3712     23580          4

In [224]:
pipeline = Pipeline(steps = [('preprocessor', feature_transformer()),
                             ('scaler', StandardScaler()), 
                             ('model', XGBRegressor(
                                           subsample= 0.9000000000000001,
                                             n_estimators= 118,
                                             min_samples_split= 20,
                                             min_samples_leaf= 12,
                                             max_depth= 9,
                                             learning_rate= 0.05,
                                             colsample_bytree= 0.4000000000000001,
                                             colsample_bylevel= 0.7,
                                             random_state = 42))
                            ])

pipeline.fit(X_df, y = y)

[06:13:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "min_samples_leaf", "min_samples_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




Pipeline(steps=[('preprocessor', feature_transformer()),
                ('scaler', StandardScaler()),
                ('model',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=0.7, colsample_bynode=1,
                              colsample_bytree=0.4000000000000001,
                              enable_categorical=False, gamma=0, gpu_id=-1,
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.05, max_delta_step=0, max_depth=9,
                              min_child_weight=1, min_samples_leaf=12,
                              min_samples_split=20, missing=nan,
                              monotone_constraints='()', n_estimators=118,
                              n_jobs=12, num_parallel_tree=1, predictor='auto',
                              random_state=42, reg_alpha=0, reg_lambda=1,
                              scale_pos_weight=1, subsample=0.900

In [250]:
pipeline['preprocessor']

feature_transformer()

In [251]:
import joblib

joblib.dump(pipeline['preprocessor'], 'feature_transformer.h5')

joblib.dump(pipeline['scaler'], 'scaler.h5')

#xgboost_model.save_model("xgboost.h5")

['scaler.h5']

### Calculate Root mean square error (to calculate confidence Range)

In [182]:
from sklearn.metrics import mean_squared_error

In [183]:
y_pred = pipeline.predict(X_df)

In [206]:
RMSE = np.sqrt(mean_squared_error(y,y_pred))
Interval = RMSE * 1.45
Interval

165.21108986579125

In [194]:
np.std(y)

698.657568740724

In [207]:
pipeline

Pipeline(steps=[('preprocessor',
                 FunctionTransformer(func=<function preprocess at 0x00000255D94EF5E0>)),
                ('scaler', StandardScaler()),
                ('model',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=0.7, colsample_bynode=1,
                              colsample_bytree=0.4000000000000001,
                              enable_categorical=False, gamma=0, gpu_id=-1,
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.05, max_delta_step=0, max_depth=9,
                              min_child_weight=1, min_samples_leaf=12,
                              min_samples_split=20, missing=nan,
                              monotone_constraints='()', n_estimators=118,
                              n_jobs=12, num_parallel_tree=1, predictor='auto',
                              random_state=42, reg_alpha=0, reg_lambda=1,


# Splitting Pipeline into multiple pipelines (For XGRegressor)

In [ ]:
import joblib

joblib.dump(pipeline, 'pipeline.h5')

In [211]:
test_pipline = pipeline

In [212]:
test_pipline.steps.pop(-1)

('model',
 XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.7,
              colsample_bynode=1, colsample_bytree=0.4000000000000001,
              enable_categorical=False, gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.05, max_delta_step=0,
              max_depth=9, min_child_weight=1, min_samples_leaf=12,
              min_samples_split=20, missing=nan, monotone_constraints='()',
              n_estimators=118, n_jobs=12, num_parallel_tree=1, predictor='auto',
              random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=0.9000000000000001, tree_method='exact',
              validate_parameters=1, verbosity=None))

In [213]:
test_pipline

Pipeline(steps=[('preprocessor',
                 FunctionTransformer(func=<function preprocess at 0x00000255D94EF5E0>)),
                ('scaler', StandardScaler())])

In [214]:
joblib.dump(test_pipline, 'pipeline2.h5')

['pipeline2.h5']

In [219]:
test_pipline.transform(X_df.loc[[0]])[0]

array([ 7.77051815, -0.5111286 , -0.29469738, -0.36835149, -0.54717014,
       -0.54477581, -0.08385255, -0.20910451, -0.13478671, -0.19666846,
       -0.07902608, -0.04829922,  2.36146789, -0.43487284, -0.32039326,
       -0.15170983,  0.73736868, -0.89037505, -0.24501611, -0.09693825,
       -0.86009213, -0.41568193, -0.08317758, -1.00270414,  7.77051815,
       -2.19232897, -0.22478823, -0.03942083, -0.1434779 , -0.19019367,
       -1.20001289,  1.3377691 ,  1.85451727,  1.60024169, -0.4170763 ,
        8.6500595 , -0.32251662,  1.12034502,  3.96502868,  5.74530219,
        1.1216814 ])

# END